In [1]:
import itertools
import os
import time
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score as auc
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook as tqdm
from sklearn.base import clone
from sklearn.metrics import accuracy_score

In [2]:
!ls

__notebook__.ipynb


In [3]:
use_sqrd_feats = False
use_combs = False
# use_percentile = True
encoder_to_use = 'target' #target or label.
# if use_sqrd_feats and use_combs:
#     raise ValueError("saiesha we talked about this. Dont use combinations and squared feats together")

In [4]:
df_train = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
df_test = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")

In [5]:
df = pd.concat([df_train,df_test], axis=0)
# df = df_train
df = df.reset_index(drop=True)
df_num = df.select_dtypes(include = float).copy(deep=True)
df_cat = df.select_dtypes(exclude = float).copy(deep=True)

In [6]:
if use_sqrd_feats:
    for i in df_num.columns:
        df_num[i+'_sqrd'] = df_num[i]**2

In [7]:
# if use_percentile:
#     for i in df_num.columns:
#         if 'sqrd' not in df.columns:

In [8]:
if use_combs:
    df_num_adds = {}
    for i,j in combinations(df_num.columns,2):
        df_num_adds[i+'_mul_' + j] = df_num[i] * df_num[j]
        df_num_adds[i+'_div_' + j] = df_num[i] / df_num[j]
        new_df = pd.DataFrame(df_num_adds)
    df_num = pd.concat([df_num,new_df],axis=1)
        

In [9]:
for i in df_num.columns:
    ma = df_num[i].replace({np.inf:-np.inf}).max()
    mi = df_num[i].replace({-np.inf:np.inf}).min()
    ma_r = 2*ma if ma>0 else -2*ma
    mi_r = 2*mi if mi < 0 else -2*mi
    df_num[i] = df_num[i].replace({np.inf:ma_r,-np.inf:mi_r})

In [10]:
if encoder_to_use == 'label':
    for i in df_cat.columns:
        le = LabelEncoder()
        df_cat[i] = le.fit_transform(df_cat[i])
elif encoder_to_use == 'target':
    for i in df_cat.columns:
        encodings = df_cat.groupby(i)['Made_Purchase'].mean()
        df_cat[i] = df_cat[i].map(encodings)
        

In [11]:
data = pd.concat([df_num,df_cat],axis=1)

In [12]:
for i in data.columns:
    if i == 'Made_Purchase':
        continue
    data[i+"_missing"] = data[i].isna()
    data[i] = data[i].fillna(data[i].mean())

In [13]:
data_train = data.iloc[:14731,:]
data_test = data.iloc[14731:,:]

In [14]:
data_train = data_train.sample(frac=1.)

In [15]:
y_train = data_train.pop('Made_Purchase')
y_train = y_train.astype(int)

In [16]:
def accuracy(estimator, X, y):
    preds = estimator.predict(X)
    out = []
    thresholds = [t/100 for t in range(38,50,1)]
    for i in thresholds:
        preds = y_preds > i
        out.append((preds == y_true.astype(bool)).sum()/len(preds))
    max_acc = max(out)
    return 'max_acc', max(out), True

def accuracy_for_grid(y_true, y_preds):
    out = []
    thresholds = [t/100 for t in range(38,50,1)]
    for i in thresholds:
        preds = y_preds > i
        out.append((preds == y_true.astype(bool)).sum()/len(preds))
    max_acc = max(out)
    return max(out)
def accuracy_for_thresholds(y_true, y_preds):
    out = []
    thresholds = [t/100 for t in range(38,50,1)]
    for i in thresholds:
        preds = y_preds > i
        out.append(((preds == y_true.astype(bool)).sum()/len(preds),i))
#     print(out)
    t_out = sorted(out,key=lambda x: x[0], reverse=True)[0]
    return t_out
grid_scorer = make_scorer(accuracy_for_grid)

## Define CV function 

In [17]:
def run_cv(est, X, y):
    skf = StratifiedKFold(shuffle = True, n_splits = 5, random_state = 1729)
    out = []
    for tr_idx, val_idx in skf.split(X, y):
        _est = clone(est)
        X_tr = X[tr_idx,:]
        y_tr = y[tr_idx]
        X_val = X[val_idx,:]
        y_val = y[val_idx]
        _est.fit(X_tr,y_tr)
        acc = accuracy_for_thresholds(y_val,_est.predict_proba(X_val)[:,1])
        out.append((_est,acc))
    return out

In [18]:
df = pd.read_csv('/kaggle/input/lgbm-044/lgbm_from_optuna_predictions0.44.csv')

In [19]:
df = df.drop(columns=['Unnamed: 0'], axis=1)

In [20]:
df = df.rename(columns= {'preds': 'Made_Purchase'})

In [21]:
df

,id,Made_Purchase
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False
...,...,...
6594,6594,False
6595,6595,False
6596,6596,False
6597,6597,False


In [22]:
df.to_csv('LGBM_from_Optuna_0.44.csv', index=False)